In [4]:
import os
import numpy as np
import pandas as pd
import geopandas as gpd
import seaborn as sns
import matplotlib.pyplot as plt

sns.set_style('white')
sns.set(font_scale=1.5)

data_dir = r'C:\Users\James\toronto-open-data-portal'

nbrhood_shp_path = os.path.join(data_dir, 'toronto-nbrhoods-wgs84', 'Neighbourhoods.shp')
nbrhood_covid_count_path = os.path.join(data_dir, 'toronto-covid-nbrhood-count-060820.csv')
nbrhood_covid_rate_path = os.path.join(data_dir, 'toronto-covid-nbrhood-rate-060820.csv')


### Reading in Non-geographic Data

**1. Neighbourhood profiles:** demographic information about the 140 Toronto Social Planning Neighbourhoods. Data is from the 2016 Census


**2. Covid19 Counts per Neighbourhood:** number of cases in each of the 14 Toronto Social Planning Neighbourhoods, downloaded 06-08-2020

In [125]:
# reading in the demographic info per neighbourhood

demo_profiles = pd.read_csv(os.path.join(data_dir, 'toronto-nbrhoods-wgs84', 'nbrhood-profiles-2016.csv'),
                            encoding='ISO-8859-1')


In [126]:
demo_profiles

,_id,Category,Topic,Data Source,Characteristic,City of Toronto,Agincourt North,Agincourt South-Malvern West,Alderwood,Annex,...,Willowdale West,Willowridge-Martingrove-Richview,Woburn,Woodbine Corridor,Woodbine-Lumsden,Wychwood,Yonge-Eglinton,Yonge-St.Clair,York University Heights,Yorkdale-Glen Park
0,1,Neighbourhood Information,Neighbourhood Information,City of Toronto,Neighbourhood Number,NaN,129,128,20,95,...,37,7,137,64,60,94,100,97,27,31
1,2,Neighbourhood Information,Neighbourhood Information,City of Toronto,TSNS2020 Designation,NaN,No Designation,No Designation,No Designation,No Designation,...,No Designation,No Designation,NIA,No Designation,No Designation,No Designation,No Designation,No Designation,NIA,Emerging Neighbourhood
2,3,Population,Population and dwellings,Census Profile 98-316-X2016001,"Population, 2016","2,731,571","29,113","23,757","12,054","30,526",...,"16,936","22,156","53,485","12,541","7,865","14,349","11,817","12,528","27,593","14,804"
3,4,Population,Population and dwellings,Census Profile 98-316-X2016001,"Population, 2011","2,615,060","30,279","21,988","11,904","29,177",...,"15,004","21,343","53,350","11,703","7,826","13,986","10,578","11,652","27,713","14,687"
4,5,Population,Population and dwellings,Census Profile 98-316-X2016001,Population Change 2011-2016,4.50%,-3.90%,8.00%,1.30%,4.60%,...,12.90%,3.80%,0.30%,7.20%,0.50%,2.60%,11.70%,7.50%,-0.40%,0.80%
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2378,2379,Mobility,Mobility status - Place of residence 5 years ago,Census Profile 98-316-X2016001,Migrants,"400,950","3,170","3,145",925,"6,390",...,"3,765","2,270","7,260",985,620,"1,350","2,425","2,310","4,965","1,345"
2379,2380,Mobility,Mobility status - Place of residence 5 years ago,Census Profile 98-316-X2016001,Internal migrants,"184,120",880,980,680,"3,930",...,"1,545","1,110","1,720",610,395,780,"1,260","1,355","1,700",580
2380,2381,Mobility,Mobility status - Place of residence 5 years ago,Census Profile 98-316-X2016001,Intraprovincial migrants,"141,135",735,760,615,"2,630",...,"1,070",960,"1,400",350,320,570,970,"1,025","1,490",445
2381,2382,Mobility,Mobility status - Place of residence 5 years ago,Census Profile 98-316-X2016001,Interprovincial migrants,"42,985",135,220,70,"1,310",...,475,150,335,250,85,210,290,325,195,135


In [127]:
profile_cols = list(zip(demo_profiles.T.loc['Category', :], demo_profiles.T.loc['Characteristic', :]))

In [143]:
profile_categories = set([i[0] for i in profile_cols])

In [147]:
for c in profile_categories:
    print(f'{c} ||| {demo_profiles[c].shape[1]} features')

Neighbourhood Information ||| 2 features
Immigration and citizenship ||| 154 features
Families, households and marital status ||| 70 features
Ethnic origin ||| 279 features
Visible minority ||| 15 features
Aboriginal peoples ||| 34 features
Language of work ||| 280 features
Mobility ||| 18 features
Income ||| 311 features
Education ||| 182 features
Labour ||| 76 features
Language ||| 808 features
Population ||| 55 features
Housing ||| 74 features
Journey to work ||| 25 features


In [128]:
demo_profiles = demo_profiles.iloc[:,6:]

In [129]:
index = pd.MultiIndex.from_tuples(profile_cols, names=['Category', 'Characteristic'])

In [131]:
demo_profiles.set_index(index, inplace=True)
demo_profiles = demo_profiles.T
demo_profiles.head()

Category                     Neighbourhood Information                       \
Characteristic                    Neighbourhood Number TSNS2020 Designation   
Agincourt North                                    129       No Designation   
Agincourt South-Malvern West                       128       No Designation   
Alderwood                                           20       No Designation   
Annex                                               95       No Designation   
Banbury-Don Mills                                   42       No Designation   

Category                           Population                   \
Characteristic               Population, 2016 Population, 2011   
Agincourt North                        29,113           30,279   
Agincourt South-Malvern West           23,757           21,988   
Alderwood                              12,054           11,904   
Annex                                  30,526           29,177   
Banbury-Don Mills                      27,695           26,918   

Category                                                  \
Characteristic               Population Change 2011-2016   
Agincourt North                                   -3.90%   
Agincourt South-Malvern West                       8.00%   
Alderwood                                          1.30%   
Annex                                              4.60%   
Banbury-Don Mills                                  2.90%   

Category                                              \
Characteristic               Total private dwellings   
Agincourt North                                9,371   
Agincourt South-Malvern West                   8,535   
Alderwood                                      4,732   
Annex                                         18,109   
Banbury-Don Mills                             12,473   

Category                                                                    \
Characteristic               Private dwellings occupied by usual residents   
Agincourt North                                                      9,120   
Agincourt South-Malvern West                                         8,136   
Alderwood                                                            4,616   
Annex                                                               15,934   
Banbury-Don Mills                                                   12,124   

Category                                                              \
Characteristic               Population density per square kilometre   
Agincourt North                                                3,929   
Agincourt South-Malvern West                                   3,034   
Alderwood                                                      2,435   
Annex                                                         10,863   
Banbury-Don Mills                                              2,775   

Category                                                     \
Characteristic               Land area in square kilometres   
Agincourt North                                        7.41   
Agincourt South-Malvern West                           7.83   
Alderwood                                              4.95   
Annex                                                  2.81   
Banbury-Don Mills                                      9.98   

Category                                            ...  \
Characteristic               Children (0-14 years)  ...   
Agincourt North                              3,840  ...   
Agincourt South-Malvern West                 3,075  ...   
Alderwood                                    1,760  ...   
Annex                                        2,360  ...   
Banbury-Don Mills                            3,605  ...   

Category                                    Mobility  \
Characteristic                     External migrants   
Agincourt North                                  605   
Agincourt South-Malvern West                     490   
Alderwood                            

In [139]:
demo_profiles['']

KeyError: 'Visible Minority'

In [16]:
nbrhood_geo_data = gpd.read_file(nbrhood_shp_path)
nbrhood_geo_cols = pd.read_csv(os.path.join(data_dir, 'toronto-nbrhoods-wgs84',
                                           'Neighbourhoods_fields.csv'), encoding='ISO-8859-1')

In [31]:
nbrhood_geo_data.rename(columns=dict(zip(nbrhood_geo_cols.field, nbrhood_geo_cols.name)), inplace=True)

In [33]:
nbrhood_geo_data.head(3)

,_id,AREA_ID,AREA_ATTR_ID,PARENT_AREA_ID,AREA_SHORT_CODE,AREA_LONG_CODE,AREA_NAME,AREA_DESC,X,Y,LONGITUDE,LATITUDE,OBJECTID,Shape__Area,Shape__Length,geometry
0,5601,25886861,25926662,49885,94,94,Wychwood (94),Wychwood (94),None,None,-79.425515,43.676919,16491505,3.217960e+06,7515.779658,"POLYGON ((-79.43592 43.68015, -79.43492 43.680..."
1,5602,25886820,25926663,49885,100,100,Yonge-Eglinton (100),Yonge-Eglinton (100),None,None,-79.403590,43.704689,16491521,3.160334e+06,7872.021074,"POLYGON ((-79.41096 43.70408, -79.40962 43.704..."
2,5603,25886834,25926664,49885,97,97,Yonge-St.Clair (97),Yonge-St.Clair (97),None,None,-79.397871,43.687859,16491537,2.222464e+06,8130.411276,"POLYGON ((-79.39119 43.68108, -79.39141 43.680..."
